In [6]:
import tensorflow as tf
from tensorflow.keras import layers, models
import xarray as xr

In [3]:
# Load data
train_ds = xr.open_dataset("/glade/scratch/lverhoef/gdl_toy_ds/train_data.nc")
validation_ds = xr.open_dataset("/glade/scratch/lverhoef/gdl_toy_ds/validation_data.nc")

In [14]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((4, 4)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1))

In [18]:
model.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mse']
)

In [ ]:
history = model.fit(
    train_ds['ellipse'], train_ds[['maj_len', 'min_len']],
    epochs=10, validation_data=(validation_ds['ellipse'], validation_ds[['maj_len', 'min_len']])
)